In [1]:
from predictor import AverageForecasts, MedianForecasts
import pandas as pd

%load_ext autoreload
%autoreload 2

In [5]:
variable_names = ["NGDP", "RGDP"]
forecast_horizons = [0, 1, 2, 3, 4]

pred1 = AverageForecasts(variable_names, forecast_horizons).fit()
pred2 = MedianForecasts(variable_names, forecast_horizons).fit()

In [8]:
mse1 = pred1.calculate_metrics()
mse2 = pred2.calculate_metrics()

In [11]:
pd.concat((mse1["NGDP"], mse2["NGDP"]), axis=1, keys=["Average", "Median"])

,Average,Median
NGDP2,402.648980,402.030728
NGDP3,455.941841,455.113778
NGDP4,477.212994,476.441524
NGDP5,507.057732,504.670070
NGDP6,539.255420,537.291887


In [12]:
pd.concat((mse1["RGDP"], mse2["RGDP"]), axis=1, keys=["Average", "Median"])

,Average,Median
RGDP2,4176.887666,4177.098634
RGDP3,4220.089875,4220.260753
RGDP4,4259.307518,4259.463319
RGDP5,4297.816958,4297.674071
RGDP6,4335.630160,4334.757780
